In [27]:
import numpy as np
import sympy as sp

In [38]:
frac = sp.Symbol('frac', nonegative=True)
etpr = sp.Symbol('etpr', nonnegative=True)
efpr = sp.Symbol('efpr', nonnegative=True)

area = sp.integrate(frac**etpr * sp.diff((frac**efpr), frac), (frac, 0, 1))

In [39]:
area

Piecewise((-0**efpr*0**etpr*efpr/(efpr + etpr) + efpr/(efpr + etpr), Ne(efpr, -etpr)), (oo*sign(efpr), True))

In [23]:
area = sp.simplify(area)

In [24]:
area

Piecewise((efpr*(1 - 0**(efpr + etpr))/(efpr + etpr), Ne(efpr, -etpr)), (Piecewise((nan, Eq(efpr, 0)), (oo, True)), True))

In [31]:
print(area)

Piecewise((efpr*(1 - 0**(efpr + etpr))/(efpr + etpr), Ne(efpr, -etpr)), (Piecewise((nan, Eq(efpr, 0)), (oo, True)), True))


In [81]:
sp.simplify(area.subs({etpr: 0.3, efpr: 2}))

0.869565217391304

In [79]:
exp_tpr = 0.3
exp_fpr = 2

x = np.linspace(0, 1, 10000)

tprs = x**exp_tpr
fprs = x**exp_fpr

auc = float(np.sum((fprs[1:] - fprs[:-1])*(tprs[:-1] + tprs[1:])/2))
auc

0.8695652145025436

In [80]:
exp_fpr / (exp_tpr + exp_fpr)

0.8695652173913044

In [68]:
frac = sp.Symbol('frac', nonegative=True)
auc = sp.Symbol('auc', nonegative=True)
alpha = sp.Symbol('alpha', nonnegative=True)
beta = sp.Symbol('beta', nonnegative=True)
p = sp.Symbol('p', nonnegative=True)
n = sp.Symbol('n', nonnegative=True)

In [69]:
accpn = p*frac**((1 - auc)*alpha/auc) + n*(1 - frac**alpha)

In [70]:
np.array([accpn.subs({frac: f, p: 10, n: 20, auc: 0.8, alpha: 0.2})/(10 + 20) for f in np.linspace(0, 1, 100)])

array([2/3, 0.665637034439202, 0.635434374165001, 0.615246351265044,
       0.599680932699825, 0.586851145330147, 0.575853861764681,
       0.566180507478589, 0.557513947937874, 0.549641978846422,
       0.542415048662417, 0.535723476628751, 0.529484240443706,
       0.523632860768368, 0.518118180465285, 0.512898875679222,
       0.507941048614652, 0.503216520880315, 0.498701594836734,
       0.494376136075483, 0.490222881470168, 0.486226908978975,
       0.482375225581157, 0.478656442918197, 0.475060519016557,
       0.471578550469244, 0.468202603617577, 0.464925576212733,
       0.461741083141571, 0.458643361330168, 0.455627190063434,
       0.452687823796560, 0.449820935164307, 0.447022566373233,
       0.444289087529673, 0.441617160740943, 0.439003709049444,
       0.436445889434129, 0.433941069252307, 0.431486805605214,
       0.429080827199514, 0.426721018348511, 0.424405404815100,
       0.422132141246011, 0.419899499985943, 0.417705861092409,
       0.415549703398796, 0.4134295

In [71]:
d1 = sp.diff(accpn, frac)
d1

-alpha*frac**alpha*n/frac + alpha*frac**(alpha*(1 - auc)/auc)*p*(1 - auc)/(auc*frac)

In [78]:
np.array([d1.subs({frac: f, p: 10, n: 20, auc: 0.7, alpha: 10}) for f in np.linspace(0, 1, 10+20)])

array([0, 0.000671430727776516, 0.00654785887595145, 0.0248130753766740,
       0.0638518898426158, 0.132901220416834, 0.241843533555318,
       0.401005040177211, 0.620875481526835, 0.911659178457388,
       1.28254349864001, 1.74053877212077, 2.28870513117193,
       2.92353754044410, 3.63123074036548, 4.38249110033322,
       5.12550254420917, 5.77658880986794, 6.20804436037780,
       6.23253129074161, 5.58335957961173, 3.88988302554514,
       0.647153185822468, -4.82112139492220, -13.4073634173279,
       -26.2738203169101, -44.9164308564974, -71.2402477223493,
       -107.647919084194, -157.142857142857], dtype=object)

In [53]:
d2 = sp.diff(d1, frac)
d2

-alpha**2*frac**alpha*n/frac**2 + alpha**2*frac**(alpha*(1 - auc)/auc)*p*(1 - auc)**2/(auc**2*frac**2) + alpha*frac**alpha*n/frac**2 - alpha*frac**(alpha*(1 - auc)/auc)*p*(1 - auc)/(auc*frac**2)

In [57]:
np.array([d2.subs({frac: f, p: 10, n: 20, auc: 0.95, alpha: 0.2}) for f in np.linspace(0, 1, 100)])

array([nan, 11538.4150037392, 3347.91580657809, 1622.38344421851,
       970.093334689229, 650.904362435696, 469.771306043600,
       356.544958838928, 280.766250596410, 227.405161463269,
       188.321735292318, 158.783250983675, 135.879457535347,
       117.737667992925, 103.106725830192, 91.1238812263612,
       81.1779477834254, 72.8257441492970, 65.7393116924246,
       59.6715746832436, 54.4334129299568, 49.8779991228591,
       45.8898780809539, 42.3772113490447, 39.2661777673625,
       36.4968695096725, 34.0202427525954, 31.7958234121287,
       29.7899610105455, 27.9744855511527, 26.3256642048234,
       24.8233834781513, 23.4505026840119, 22.1923387842450,
       21.0362528708633, 19.9713159308427, 18.9880369349189,
       18.0781402747023, 17.2343825411948, 16.4504008692791,
       15.7205867637560, 15.0399806138224, 14.4041830961679,
       13.8092804361884, 13.2517810965690, 12.7285619329631,
       12.2368222277344, 11.7740443073215, 11.3379596838093,
       10.926519849

In [49]:
d2.subs({p:10, n:20, alpha: 0.3, frac: 0.1, auc: 0.5})

105.249319061727

In [45]:
sp.solve(sp.diff(acc, frac), frac)

[(p*(1 - auc)/(auc*n))**(auc/(alpha*(2*auc - 1)))]

In [ ]:
(p*(1 - auc)/(auc*n))**(auc/(alpha*(2*auc - 1)))